<a href="https://colab.research.google.com/github/LuciaPuppo897/Data_UY/blob/main/Trabajo_final_museos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Análisis comparativo de exposiciones realizadas en los museos Azulejo, Subte y MuHar de Montevideo durante 2023 y 2024**


---


**Objetivo del proyecto**

El presente trabajo tiene como objetivo analizar la distribución y características de las exposiciones culturales desarrolladas en tres museos de Montevideo —Museo del Azulejo, Centro de Exposiciones Subte y Museo de Historia del Arte (MuHar)— durante los años 2023 y 2024.

A partir de este análisis, se pretende identificar patrones en la programación y diversidad cultural en la oferta museística de Montevideo, y generar una predicción sobre la posible oferta para el año 2025.

**Origen de los datos**

Los datos utilizados provienen del portal de Datos Abiertos de la Intendencia de Montevideo, específicamente del conjunto:
https://ckan.montevideo.gub.uy/dataset/museos

Los datasets utilizados corresponden a los museos mencionados para los años 2023 y 2024, y están disponibles en formato CSV. Consideramos que debido a la naturaleza de los dataset y el objetivo que nos propusimos, era necesario tener muestreos de 2 años como referencia, a su vez considerando que los dataset no superan los 50 datos en promedio,y que varias variables no se encuentran reflejadas en su totalidad en todos los museos, decidimos considerar para la escala de este trabajo las siguientes variables:
Se busca estudiar las exposiciones según variables como:

* Género de artistas
* Nacionalidad (local o internacional)
* Rango etario
* Público destinatario
* Tipo de actividad (exposición, conversatorio, etc.)


A continuación se subiran los dataset a ser utilizados:
* subte_2023.csv
* subte_2024.csv
* muhar_2023.csv
* muhar_2024.csv
* museo-del-azulejo_2023.csv
* museo-del-azulejo_2024.csv

Para su posterior limpieza, depuración y tratamiento se utilizara panda


In [19]:
import os
from google.colab import files

# Eliminar los archivos existentes
def eliminar_archivos_anteriores():
    archivos = os.listdir('/content/')
    archivos_a_conservar = [
        'museo-del-azulejo_2023.csv',
        'subte_2023.csv',
        'museo-del-azulejo_2024.csv',
        'subte_2024.csv',
        'muhar_2023.csv',
        'muhar_2024.csv'
    ]

    for archivo in archivos:
        archivo_path = os.path.join('/content/', archivo)
        # Verificar que es un archivo, no un directorio, y que no esté en la lista de conservación
        if archivo not in archivos_a_conservar and os.path.isfile(archivo_path):
            os.remove(archivo_path)
            print(f'Archivo {archivo} eliminado.')

# Llamar la función para eliminar los archivos anteriores
eliminar_archivos_anteriores()

# Subir los nuevos archivos CSV
uploaded = files.upload()

# Ver los archivos subidos
uploaded.keys()



Archivo muhar_2024 (1).csv eliminado.
Archivo muhar_2023 (1).csv eliminado.
Archivo subte_2023 (1).csv eliminado.
Archivo subte_2024 (1).csv eliminado.
Archivo museo-del-azulejo_2023 (1).csv eliminado.
Archivo museo-del-azulejo_2024 (1).csv eliminado.


Saving muhar_2023.csv to muhar_2023 (1).csv
Saving subte_2023.csv to subte_2023 (1).csv
Saving museo-del-azulejo_2024.csv to museo-del-azulejo_2024 (1).csv
Saving museo-del-azulejo_2023.csv to museo-del-azulejo_2023 (1).csv
Saving muhar_2024.csv to muhar_2024 (1).csv
Saving subte_2024.csv to subte_2024 (1).csv


dict_keys(['muhar_2023 (1).csv', 'subte_2023 (1).csv', 'museo-del-azulejo_2024 (1).csv', 'museo-del-azulejo_2023 (1).csv', 'muhar_2024 (1).csv', 'subte_2024 (1).csv'])

Se cambió el sufijo (1) en los archivos para hacerlos más comprensibles y facilitar su posterior referencia en el análisis.

In [22]:
import os

# Lista de archivos a conservar
archivos_a_conservar = [
    'museo-del-azulejo_2023.csv',
    'subte_2023.csv',
    'museo-del-azulejo_2024.csv',
    'subte_2024.csv',
    'muhar_2023.csv',
    'muhar_2024.csv'
]

# Listar los archivos en el directorio actual
archivos = os.listdir('/content/')

# Eliminar archivos que no están en la lista de conservación
for archivo in archivos:
    archivo_path = os.path.join('/content/', archivo)
    # Verificar que es un archivo y no un directorio
    if os.path.isfile(archivo_path) and archivo not in archivos_a_conservar:
        os.remove(archivo_path)
        print(f'Archivo {archivo} eliminado.')

# Verificar que solo queden los archivos correctos
os.listdir('/content/')



Archivo muhar_2024 .csv eliminado.
Archivo subte_2023 .csv eliminado.
Archivo subte_2024 .csv eliminado.
Archivo museo-del-azulejo_2024 .csv eliminado.
Archivo museo-del-azulejo_2023 .csv eliminado.
Archivo muhar_2023 .csv eliminado.


['.config',
 'subte_2024.csv',
 'museo-del-azulejo_2023.csv',
 'subte_2023.csv',
 'muhar_2024.csv',
 'muhar_2023.csv',
 'museo-del-azulejo_2024.csv',
 'sample_data']

**Limpieza de Datos:**

Antes de realizar cualquier análisis, fue necesario llevar a cabo una limpieza y estandarización de los datos debido a las inconsistencias encontradas en los diferentes conjuntos de datos.
**Limitaciones:** Es importante señalar que los conjuntos de datos utilizados presentan diferencias en sus formatos y valores. Algunos archivos incluyen tanto el mes de inicio como el mes de finalización de las actividades, mientras que otros solo contienen la fecha, o no contienen una de las dos claves (inicio o fin) Debido a esta discrepancia, se decidió no tomar en cuenta la duración de las actividades, ya que los valores inconsistentes dificultan el cálculo preciso de este parámetro.

Transformaciones aplicadas:
* Conversión de Meses: En un inicio se intento calcular la duración de las actividades, como se describio previamente esto fue descartado.

* Estandarización de Categorías: Para otras columnas con valores categóricos, como público_destinatario y Actividades, también se realizó un proceso de estandarización. Esto implicó unificar las distintas formas en que se escribían los valores (por ejemplo, "todo público", "adolescentes", etc.), para asegurar que las categorías fueran consistentes a través de todos los datasets.

* Manejo de Valores Nulos y Erróneos: Se eliminaron o imputaron los valores nulos en las columnas clave, como Año, Museo y Actividades, para asegurar que los análisis posteriores fueran precisos y no estuvieran sesgados por la falta de datos.Por ejemplo ciertos dataset no contaban con la columna “museo” pero gracias a que se sabe a qué museo pertenecían se agrego para poder hacer la comparación más fácil. Con la columna *Actividades* se tuvo que tomar en cuenta que en algunos dataset se le denominaba *Actividad*
